In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

# Inicia o navegador e cria a variável 'driver' na memória
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Abre o site (pode ser a home ou a página de login)
driver.get("https://seller.shopee.com.br/portal/settings/shop/rating?pageNumber=2&fromPageNumber=3&cursor=70370359730828&pageSize=20&replied=REPLIED&ratingStar=5&ctimeStart=1761966000000&ctimeEnd=1762052399000") 
print("Navegador aberto! Agora faça o login manualmente ou via código.")

Navegador aberto! Agora faça o login manualmente ou via código.


In [2]:
def raspar_dados_da_tela(driver_ativo):
    print(" -> Lendo dados da página atual...")
    dados = []
    
    # Classes CSS (Verifique se continuam as mesmas inspecionando o site)
    CSS_NOME = ".ml-2"
    CSS_PRODUTO = ".min-w-0.font-medium" 
    CSS_DATA = ".text-xs"
    CSS_COMENTARIO = ".break-all.whitespace-pre-wrap"
    CSS_RESPOSTA = ".mt-2.leading-4"

    # Encontra elementos
    elementos_nomes = driver_ativo.find_elements(By.CSS_SELECTOR, CSS_NOME)
    
    if len(elementos_nomes) == 0:
        print("    ⚠️ Nenhum item encontrado nesta página (ou classes mudaram).")
        return pd.DataFrame()

    for el_nome in elementos_nomes:
        item = {}
        try:
            item['Nome'] = el_nome.text
            
            # Sobe para o card principal (Bisavô) usando XPath relativo
            card = el_nome.find_element(By.XPATH, "./../../..") 
            
            try:
                item['Produto'] = card.find_element(By.CSS_SELECTOR, CSS_PRODUTO).text
            except: item['Produto'] = "Erro Prod"
                
            try:
                t_data = card.find_element(By.CSS_SELECTOR, CSS_DATA).text
                item['Data'] = t_data.split(' ')[0]
            except: item['Data'] = "Erro Data"
                
            try:
                # Pega o texto bruto
                texto_comentario = card.find_element(By.CSS_SELECTOR, CSS_COMENTARIO).text
                # CORREÇÃO: Substitui quebras de linha (\n) por um separador seguro ( | )
                # Isso impede que o CSV quebre em novas linhas
                item['Comentario'] = texto_comentario.replace('\n', ' | ').replace('\r', '')
            except: item['Comentario'] = ""
                
            try:
                resp = card.find_element(By.CSS_SELECTOR, CSS_RESPOSTA).text
                texto_limpo = resp.replace("Resposta do Vendedor:", "").strip()
                # CORREÇÃO: Também remove quebras de linha da resposta
                item['Resposta Vendedor'] = texto_limpo.replace('\n', ' | ').replace('\r', '')
            except: item['Resposta Vendedor'] = "Sem resposta"
            
            dados.append(item)
            
        except Exception as e:
            continue # Pula item com erro grave e vai para o próximo

    return pd.DataFrame(dados)

In [3]:
def ir_para_proxima_pagina(driver_ativo):
    """
    Tenta encontrar e clicar no botão de próxima página (>).
    Usa JavaScript forçado para garantir que o clique funcione em botões React/EDS.
    """
    print(" -> Tentando mudar de página...")
    
    # LISTA DE ESTRATÉGIAS (XPATHS)
    seletores_possiveis = [
        "//button[contains(@class, 'eds-react-pagination-pager__button-next')]", # CLASSE EXATA
        "//button[.//span[@aria-label='arrow-right-bold']]", # Pelo ícone
        "//button[contains(@class, 'shopee-icon-button--right')]" # Antigo
    ]

    botao_encontrado = None

    for xpath in seletores_possiveis:
        try:
            elementos = driver_ativo.find_elements(By.XPATH, xpath)
            for el in elementos:
                if el.is_displayed():
                    botao_encontrado = el
                    break
            if botao_encontrado:
                print(f" -> Botão encontrado: {xpath}")
                break
        except:
            continue

    if not botao_encontrado:
        print(" -> ❌ Botão 'Próximo' não encontrado visualmente. Parando código.")
        return False

    # --- VERIFICAÇÃO RIGOROSA DE BOTÃO DESATIVADO ---
    # Verifica atributo padrão, classe CSS e aria-disabled
    classes_botao = botao_encontrado.get_attribute("class") or ""
    aria_disabled = botao_encontrado.get_attribute("aria-disabled")
    is_disabled_attr = botao_encontrado.get_attribute("disabled")

    if (is_disabled_attr is not None) or \
       ("disabled" in classes_botao.lower()) or \
       (aria_disabled == "true"):
        print(" -> Botão 'Próximo' detectado como DESATIVADO (Fim da paginação).")
        return False

    # CLIQUE FORÇADO VIA JAVASCRIPT (Mais robusto para Shopee)
    try:
        driver_ativo.execute_script("arguments[0].click();", botao_encontrado)
        return True
    except Exception as e:
        print(f" -> Erro crítico ao clicar: {e}")
        return False

In [4]:
# --- CONFIGURAÇÃO ---
NUMERO_TOTAL_PAGINAS = 4  # <--- DIGITE AQUI O TOTAL DE PÁGINAS QUE EXISTEM
# --------------------

lista_de_dataframes = []
ultimo_primeiro_nome = None 

print(f"Iniciando varredura fixa de {NUMERO_TOTAL_PAGINAS} páginas...")

try:
    # Loop 'for' garante que rodará exatamente o número de vezes pedido
    for i in range(NUMERO_TOTAL_PAGINAS):
        pagina_atual = i + 1
        print(f"\n--- Processando Página {pagina_atual} de {NUMERO_TOTAL_PAGINAS} ---")
        
        # 1. Raspa a página
        df_parcial = raspar_dados_da_tela(driver)
        
        # Retry logic: Se vier vazia, tenta esperar e ler de novo
        if df_parcial.empty:
            print(" -> Leitura vazia. Tentando novamente em 3 segundos...")
            time.sleep(3)
            df_parcial = raspar_dados_da_tela(driver)
        
        # Salva os dados se conseguiu ler
        if not df_parcial.empty:
            lista_de_dataframes.append(df_parcial)
            print(f" -> {len(df_parcial)} avaliações coletadas.")
            # Pega o primeiro nome para verificar transição depois
            nome_atual_topo = df_parcial.iloc[0]['Nome']
        else:
            print(" -> Página vazia ou erro de carregamento.")
            nome_atual_topo = None

        # 2. Verifica se precisa avançar (Não clica se for a última página)
        if pagina_atual < NUMERO_TOTAL_PAGINAS:
            
            sucesso_clique = ir_para_proxima_pagina(driver)
            
            if not sucesso_clique:
                print(" -> Não foi possível clicar na próxima página. Parando antes do previsto.")
                break
                
            # 3. Espera a página mudar visualmente (para evitar duplicidade na próxima leitura)
            print(" -> Aguardando atualização da tabela...")
            if nome_atual_topo:
                mudou = False
                for _ in range(15): # Espera até 15 segundos
                    time.sleep(1)
                    try:
                        topo_agora = driver.find_element(By.CSS_SELECTOR, ".ml-2").text
                        if topo_agora != nome_atual_topo:
                            print(" -> Página mudou visualmente!")
                            mudou = True
                            break
                    except: pass
                if not mudou:
                    print(" ⚠️ Aviso: A tabela não mudou visualmente. Próxima leitura pode ser duplicada.")
            else:
                time.sleep(5) # Fallback simples se não tiver nomes para comparar

except KeyboardInterrupt:
    print("\n🛑 Interrupção manual!")

print("Varredura finalizada! Execute a próxima célula para salvar.")

Iniciando varredura fixa de 4 páginas...

--- Processando Página 1 de 4 ---
 -> Lendo dados da página atual...
 -> 25 avaliações coletadas.
 -> Tentando mudar de página...
 -> Botão encontrado: //button[contains(@class, 'eds-react-pagination-pager__button-next')]
 -> Aguardando atualização da tabela...

--- Processando Página 2 de 4 ---
 -> Lendo dados da página atual...
 -> 25 avaliações coletadas.
 -> Tentando mudar de página...
 -> Botão encontrado: //button[contains(@class, 'eds-react-pagination-pager__button-next')]
 -> Aguardando atualização da tabela...

--- Processando Página 3 de 4 ---
 -> Lendo dados da página atual...
 -> 25 avaliações coletadas.
 -> Tentando mudar de página...
 -> Botão encontrado: //button[contains(@class, 'eds-react-pagination-pager__button-next')]
 -> Aguardando atualização da tabela...

--- Processando Página 4 de 4 ---
 -> Lendo dados da página atual...
 -> 18 avaliações coletadas.
Varredura finalizada! Execute a próxima célula para salvar.


In [5]:
if lista_de_dataframes:
    # Junta todos os pedaços em um único DataFrame
    df_final = pd.concat(lista_de_dataframes, ignore_index=True)
    
    # Remove duplicatas exatas se houver (segurança extra)
    df_final = df_final.drop_duplicates()
    
    display(df_final.head())
    print(f"Total de avaliações coletadas (sem duplicatas): {len(df_final)}")
    
    # Salva em CSV
    nome_arquivo = f"shopee_reviews_total_{len(df_final)}.csv"
    df_final.to_csv(nome_arquivo, index=False, encoding='utf-8-sig')
    print(f"Arquivo salvo com sucesso: {nome_arquivo}")
else:
    print("Nenhum dado foi coletado. Verifique se o login foi feito corretamente.")


,Nome,Produto,Data,Comentario,Resposta Vendedor
0,,Erro Prod,vs,,Sem resposta
3,Ver,Erro Prod,Avaliações,,Sem resposta
5,9k74u5nnj2,"Teclado Mecânico Gamer SuperFrame Player, RGB,...",03/12/2025,,Sua avaliação é muito importante e somos muito...
6,thiagoxd,"Pasta Térmica Arctic MX-6, 4G, 7.5W/mK, ACTCP0...",03/12/2025,Muito bom,Sua avaliação é muito importante e somos muito...
7,otavio567,"Memória DDR5 XPG Lancer Blade RGB, 16GB, 6000M...",03/12/2025,Não as coloquei ainda mais são igual as do anú...,Muito obrigado pela sua avaliação positiva! 🌟 ...


Total de avaliações coletadas (sem duplicatas): 75
Arquivo salvo com sucesso: shopee_reviews_total_75.csv
